In [48]:
from spotify_infos import sp, me

Ajouter une playlist

In [49]:
playlist = sp.user_playlist_create(
    user=me["id"],
    name="Mood2Music - Demo",
    public=False,
    description="Playlist generated during hackathon"
)
print("Playlist:", playlist["external_urls"]["spotify"])

Playlist: https://open.spotify.com/playlist/6d3z6sHGUc9T2LgqOuqxwb


In [50]:
print(playlist["id"])

6d3z6sHGUc9T2LgqOuqxwb


In [51]:
print(playlist)

{'collaborative': False, 'description': 'Playlist generated during hackathon', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6d3z6sHGUc9T2LgqOuqxwb'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/6d3z6sHGUc9T2LgqOuqxwb', 'id': '6d3z6sHGUc9T2LgqOuqxwb', 'images': [], 'primary_color': None, 'name': 'Mood2Music - Demo', 'type': 'playlist', 'uri': 'spotify:playlist:6d3z6sHGUc9T2LgqOuqxwb', 'owner': {'href': 'https://api.spotify.com/v1/users/thibaultm2003', 'id': 'thibaultm2003', 'type': 'user', 'uri': 'spotify:user:thibaultm2003', 'display_name': None, 'external_urls': {'spotify': 'https://open.spotify.com/user/thibaultm2003'}}, 'public': False, 'snapshot_id': 'AAABgYYGOz0/tc/C1z/Ch8+6h+DqmXSL', 'tracks': {'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'href': 'https://api.spotify.com/v1/playlists/6d3z6sHGUc9T2LgqOuqxwb/tracks', 'total': 0, 'items': []}}


In [52]:
sp.playlist_add_items(playlist["id"], [mhd_id])

{'snapshot_id': 'AAAAAi3SaRhlQxx0uwO7hH6axpgqeFQg'}

Ajouter un son à une playlist

In [53]:
sp.playlist_add_items(playlist["id"], ["spotify:track:4uLU6hMCjMI75M1A2tKUQC"])
print("Added one track.")

Added one track.


In [54]:
# --- ajouter un titre précis à une playlist ---
def findsongID(query : str, type = 'track'):
    results = sp.search(q = query, limit=1, type = type)
    song_id = results[type + 's']['items'][0]['id']
    return song_id

def findplaylistID(query : str, type = 'playlist'):
    results = sp.current_user_playlists(limit=50)
    for pl in results["items"]:
        if query.lower() in pl["name"].lower():
            return pl["id"]
        
def add2playlist(query_song : str, query_playlist : str, type = 'track'):
    song_id, playlist_id = findsongID(query_song), findplaylistID(query_playlist)
    sp.playlist_add_items(playlist_id, [f"spotify:track:{song_id}"])
    return (f"I added {query_song} to your playlist {query_playlist} !")

In [59]:
def _find_song_id(query: str, market: Optional[str] = None) -> Optional[str]:
    res = sp.search(q=query, limit=1, type="track", market=market)
    items = res.get("tracks", {}).get("items", []) or []
    return items[0]["id"] if items else None


def _find_my_playlist_id(query: str) -> Optional[str]:
    res = sp.current_user_playlists(limit=50)
    for pl in res.get("items", []):
        name = (pl.get("name") or "").lower()
        if query.lower() in name and pl.get("owner", {}).get("id") == me["id"]:
            return pl["id"]
    return None

def add2playlist(song_query: str, playlist_query: str, allow_duplicates: bool = True, market: Optional[str] = "from_token") -> Dict[str, Any]:
    """
    Find best match for 'song_query' and add it to YOUR playlist whose name contains 'playlist_query'.
    Scopes: playlist-read-private + playlist-modify-private (or public)
    """
    song_id = _find_song_id(song_query, market=market)
    if not song_id:
        return {"status": "not_found", "entity": "track", "query": song_query}

    playlist_id = _find_my_playlist_id(playlist_query)
    if not playlist_id:
        return {"status": "not_found", "entity": "playlist", "query": playlist_query}

    track_uri = f"spotify:track:{song_id}"

    if not allow_duplicates:
        items = sp.playlist_items(playlist_id, limit=100).get("items", [])
        if any(track_uri == (it.get("track") or {}).get("uri") for it in items):
            return {"status": "skipped", "reason": "duplicate", "playlist_id": playlist_id, "track_uri": track_uri}

    sp.playlist_add_items(playlist_id, [track_uri])
    pl = sp.playlist(playlist_id, fields="name,external_urls.spotify")
    return {"status": "added", "playlist_id": playlist_id, "playlist_name": pl["name"], "playlist_url": pl["external_urls"]["spotify"], "track_uri": track_uri}


In [62]:
add2playlist("DTMF", "La putain de ta mère")

{'status': 'added',
 'playlist_id': '3ssN4qLkzvVsDeEnb9wpDZ',
 'playlist_name': 'La putain de ta mère',
 'playlist_url': 'https://open.spotify.com/playlist/3ssN4qLkzvVsDeEnb9wpDZ',
 'track_uri': 'spotify:track:3sK8wGT43QFpWrvNQsrQya'}

In [30]:
findsongID("afrotrap la puissance")

'6wdI9sg2VdgjqSg0ozu9PK'

In [31]:
findplaylistID("Afrotrap MHD")

'6jjAbvUQC5RifW8CNAItTe'

In [35]:
sp.playlist_add_items("6jjAbvUQC5RifW8CNAItTe", ["spotify:track:6wdI9sg2VdgjqSg0ozu9PK"])

HTTP Error for POST to https://api.spotify.com/v1/playlists/6jjAbvUQC5RifW8CNAItTe/tracks with Params: {'position': None} returned 403 due to Forbidden


SpotifyException: http status: 403, code: -1 - https://api.spotify.com/v1/playlists/6jjAbvUQC5RifW8CNAItTe/tracks:
 Forbidden, reason: None

Trouver son

In [19]:
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [21]:
def search_id(query : str, playlist_name : str, type = 'track'):
    results = sp.search(q = query, limit=1, type = type)
    id = results[type + 's']['items'][0]['id']
    return id


In [26]:
print(search_id("mhd afro trap", "mhd"))

mhd_id = search_id("mhd afro trap", "mhd")

6wdI9sg2VdgjqSg0ozu9PK
